# Data Processing - Linear and Non Linear regression and  Residuals

### Import Modules 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#Linear Regression 
from sklearn.linear_model import LinearRegression

# Import curve fitting package from scipy
from scipy.optimize import curve_fit

### Import data

In [4]:
dataframe = pd.read_excel("Insert File Path.xlsx",
                          sheet_name='Insert sheet name ',
                          skiprows=0)

#Name of Control Group
Control_name = 'Control Group'

#List with group names
fly_group = ('Group1','Group2','Group3')


#List with motor parameters  
motor_param  = ['freq','sw v(AVR)','sw v(F)','sw v(M)','sw v(H)','sw s(AVR)',
                    'sw s(F)','sw s(M)','sw s(H)','sw t(AVR)','sw t(SD)','sw t(F)','sw t(M)','sw t(H)','stc t(AVR)',
                    'stc t(F)','stc t(M)','stc t(H)','stnc stabl',
                    'SD AEP','SD PEP','F_AEP STD','M_AEP STD','H_AEP STD','F_PEP STD','M_PEP STD','H_PEP STD',
                     'Stc Str','Stc Str F','Stc Str M','Stc Str H','trip index','tetrapod index','non-can.','wave index','duty Fact',
                    'tTripod','tTransition']


## Linear and non linear regression functions

In [5]:
#Define Linear regression Functions 
def Lregression(dataframe, Control_name, motor_param):
    """
    :param dataframe: N x P data matrix where N is number of features and P number of data points
    :param Control_name:str name of the control group
    :param motor_param: str name of the motor paramater
    
    :return:model: Linear regression model 
            residuals_control:array containing residuals for each fly in the selected fly group
            num_flies: int number of flies
    """
    #Linear Regression for the Control and Residuals for control
    df = dataframe.set_index('condition') #transform data into pandas organized by condition
    df_x= df['speed'].loc[Control_name] # select our independent variable: speed for the control
    num_flies = (len(df_x)) #important parameter to define the results matrix later
    x = df_x.to_numpy()[:, None]       #organize data in a vector for linear regression
    df_y = df[motor_param].loc[Control_name]  #select dependent variable: the motor parameter for the control
    y = df_y.to_numpy()[:, None]


    model = LinearRegression().fit(x,y)  #Perform the linear regression model fitted to the control 
    y_pred = model.predict(x)            #For the x valies draw the linear regression line
    residuals_control = y-y_pred         # Residuals for the control
    
    return model, residuals_control, num_flies


def make_residuals(model, motor_param, fly_group):
    """
    :param model: Linear Regression model
    :param motor_param: str name of the motor paramater
    :param fly_group: str name of the fly group
    
    :return: num_flies: int number of flies, 
            residuals_groups: array containing residuals for each fly in the selected fly group
    """
    
    df = dataframe.set_index('condition') 
    df_x= df['speed'].loc[fly_group]
    num_flies = len(df_x)
    x_group1 = df_x.to_numpy()[:, None]
    df_y = df[motor_param].loc[fly_group]
    y_group1 = df_y.to_numpy()[:, None]

    y_pred1 = model.predict(x_group1)
    residuals_groups = y_group1-y_pred1

    return num_flies, residuals_groups


In [6]:
# Logarithmic Regression Functions 

def Logregression(dataframe, Control_name, motor_param):
    '''This funtion does a Logarithmic Regression which is a Non-linear Regression the function is y= a*log(x)+b
      :param dataframe:  N x P data matrix where N is number of features and P number of data points
      :param Control_name:str  name of the control group
      :param motor_param:str  name of the motor parameter
      
      :return: pars: array [a,b] constants of logarithmic function: a*np.log(x)+b 
              residuals_control:array containing residuals for each fly in the selected fly group 
              num_flies: int number of flies 

    '''
    #Linear Regression for the Control and Residuals for control
    df = dataframe.set_index('condition') #transform data into pandas organized by condition
    df_x= df['speed'].loc[Control_name] # select our independent variable: speed for the control
    num_flies = (len(df_x)) #important parameter to define the results matrix later
    x = df_x.to_numpy()      
    df_y = df[motor_param].loc[Control_name]  #select dependent variable: the motor parameter for the control
    y = df_y.to_numpy()


    # Function to calculate the exponential with constants a and b
    def log(x, a, b):
        return a*np.log(x)+b
    
    # Fit the exponential data
    pars, cov = curve_fit(f=log, xdata=x, ydata=y, p0=[-1, 0], bounds=(-np.inf, np.inf))
    # Get the standard deviations of the parameters (square roots of the # diagonal of the covariance)
    stdevs = np.sqrt(np.diag(cov))

    # Calculate the residuals
    residuals_control = y - log(x, *pars) # Residuals for the control
    
    #Calculate R squared
    ss_res = np.sum(residuals_control**2)
    ss_tot = np.sum((y-np.mean(y))**2)
    r_squared = 1 - (ss_res / ss_tot)
    
    
    return pars, residuals_control, num_flies



def make_logresiduals(pars, motor_param, fly_group):
    """
    :param pars: array [a,b] constants of logarithmic function: a*np.log(x)+b defined in previous function
    :param motor_param: string  name of the motor paramater
    :param fly_group: string name of the fly group
    
    :return: num_flies: int number of flies, 
            residuals_groups: array containing residuals for each fly in the selected fly group
 
            
    """
    
    df = dataframe.set_index('condition') 
    df_x= df['speed'].loc[fly_group]
    num_flies = len(df_x)
    x_group1 = df_x.to_numpy()
    df_y = df[motor_param].loc[fly_group]
    y_group1 = df_y.to_numpy()
    
    # Import curve fitting package from scipy
    from scipy.optimize import curve_fit
    # Function to calculate the exponential with constants a and b
    def log(x, a, b):
        return a*np.log(x)+b
    residuals_groups = y_group1 - log(x_group1, *pars)

    return num_flies, residuals_groups



In [7]:
# Power Regression Functions 

def Power_regression(dataframe, Control_name, motor_param):
    '''This funtion does a Power Regression which is a Non-linear Regression the function is y= a*x^b
      :param dataframe:  N x P data matrix where N is number of features and P number of data points
      :param Control_name:str name of the control group
      :param motor_param:str name of the motor parameter
    
      :return: pars: array [a,b] constants of power regression: a*np.power(x, b) 
              residuals_control:array containing residuals for each fly in the selected fly group 
              num_flies: int number of flies
             
    '''
    #Linear Regression for the Control and Residuals for control
    df = dataframe.set_index('condition') #transform data into pandas organized by condition
    df_x= df['speed'].loc[Control_name] # select our independent variable speed for the control
    num_flies = (len(df_x)) #important parameter to define the results matrix later
    x = df_x.to_numpy()      
    df_y = df[motor_param].loc[Control_name]  #select dependent variable the motor parameter for the control
    y = df_y.to_numpy()
    #plt.plot(x, y,'o')


    # Function to calculate the power-law with constants a and b
    def power_law(x, a, b):
        return a*np.power(x, b)

    # Fit the power-law data
    pars, cov = curve_fit(f=power_law, xdata=x, ydata=y, p0=[-1, 0], bounds=(-np.inf, np.inf))
    # Get the standard deviations of the parameters (square roots of the # diagonal of the covariance)
    stdevs = np.sqrt(np.diag(cov))
    # Calculate the residuals
    residuals_control = y - power_law(x, *pars)
    
    
    return pars, residuals_control, num_flies

def make_powresiduals(pars, motor_param, fly_group):
    '''
    :param: pars: array [a,b] constants of power regression: a*np.power(x, b) defined in previous function
    :param motor_param: string name of the motor parameter
    :param fly_group: string name of the fly group
    
    :return: num_flies: int number of flies
             residuals_groups: array containing residuals for each fly in the selected fly group 
    
    '''
    df = dataframe.set_index('condition') 
    df_x= df['speed'].loc[fly_group]
    num_flies = len(df_x)
    x_group1 = df_x.to_numpy()
    df_y = df[motor_param].loc[fly_group]
    y_group1 = df_y.to_numpy()
    

    # Function to calculate the power-law with constants a and b
    def power_law(x, a, b):
        return a*np.power(x, b)

    residuals_groups = y_group1 - power_law(x_group1, *pars)

    return num_flies, residuals_groups



## Linear and Non-Linear Regression for all the motor parameters for Control 


In [8]:
#Loop over motor parameter to get the model for the control and residuals

#Determine the number of flies in each group
num_flies_group = dataframe.groupby('condition',sort=False).size().values  
num_flies_Control = num_flies_group[0]  #Select the first control group

#Empty matrix NxP where N is the number of flies P the number of parameters 
Control_res = np.empty((num_flies_Control, len(motor_param))) 

#for loop to get the residuals for each motor parameter organized into columns 
for i, param in enumerate(motor_param):
    
    if param == 'period' or param == 'stc t(AVR)':
        pars, residuals_control,num_flies = Power_regression(dataframe, Control_name, param)
        Control_res[:, i] = residuals_control.ravel()  #Each column correspond to a specific motor parameter
    elif param == 'stnc stabl' or param == 'Overall Stc Str':
        pars, residuals_control,num_flies = Logregression(dataframe, Control_name,param)
        Control_res[:, i] = residuals_control.ravel()  #Each column correspond to a specific motor parameter 
    else: 
        model, residuals_control, num_flies = Lregression(dataframe, Control_name, param)
        Control_res[:, i] = residuals_control.ravel()  #Each column correspond to a specific motor parameter 



## Linear and non linear Regression for the different groups and motor parameters


In [12]:
#loop over fly groups and motor parameters to fill the matrix 
all_groups = []
for i,group in enumerate(fly_group):
    #Get the num_flies for the matrix
    num_flies, residuals_groups = make_residuals(model, 'speed', group)
    #Make a empty matrix NxP where N is the number of flies P the number of parameters 
    Group_res = np.zeros((num_flies, len(motor_param)))
    for i, param in enumerate(motor_param):
        if param == 'period' or param == 'stc t(AVR)':
            pars, residuals_control,num_flies = Power_regression(dataframe, Control_name, param)
            num_flies, residuals_groups = make_powresiduals(pars, param, group)
            Group_res[:, i] = residuals_groups       
        elif param == 'stnc stabl' or param == 'Overall Stc Str':
            pars, residuals_control,num_flies = Logregression(dataframe, Control_name,param)
            num_flies, residuals_groups = make_logresiduals(pars, param, group)
            Group_res[:, i] = residuals_groups.ravel() 
        else: 
            model, residuals_control, num_flies = Lregression(dataframe, Control_name,param)
            num_flies, residuals_groups = make_residuals(model, param, group)
            Group_res[:, i] = residuals_groups.ravel()

    all_groups.append(Group_res)

    
result_arr = np.concatenate(all_groups)  
results_C = np.vstack((Control_res,result_arr))#Merge control Matrix with groups matrix



## Organizing the data and saving to excel file


In [13]:

Results = pd.DataFrame(results_C)

# Change the column names 
Results.columns =motor_param

# Change the row indexes 
num_flies_group = dataframe.groupby('condition',sort=False).size().values 
groups_size = num_flies_group[1:] #size of the groups excluding the control

#make a list with all the names of the non control groups repeated the number of times in each group
groups_list_raw= []
for i in range (len(groups_size)):
    x = [fly_group[i]]*groups_size[i]
    groups_list_raw.append(x)

groups_list = [item for sublist in groups_list_raw for item in sublist] #compreension for the lists

#append a list with the control 
control_list = [Control_name]*num_flies_group[0]

#List Combining control and groups
list_all = control_list +groups_list
Results.index = list_all


Results.to_csv('Residuals_python',index=True)
Results.to_excel('Residuals_python.xlsx')